In [13]:
import os
import sys
sys.path.append(os.path.join(os.path.dirname(os.path.realpath("__file__")), "pytorch_geometric_temporal"))
# https://stackoverflow.com/questions/35569042/ssl-certificate-verify-failed-with-python3
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
pip install torch_geometric

  Using cached torch_geometric-2.6.1-py3-none-any.whl.metadata (63 kB)
  Using cached fsspec-2024.10.0-py3-none-any.whl.metadata (11 kB)
  Using cached aiohappyeyeballs-2.4.3-py3-none-any.whl.metadata (6.1 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached attrs-24.2.0-py3-none-any.whl.metadata (11 kB)
Using cached torch_geometric-2.6.1-py3-none-any.whl (1.1 MB)
Using cached fsspec-2024.10.0-py3-none-any.whl (179 kB)
Using cached aiohappyeyeballs-2.4.3-py3-none-any.whl (14 kB)
Using cached aiosignal-1.3.1-py3-none-any.whl (7.6 kB)
Using cached attrs-24.2.0-py3-none-any.whl (63 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1 requires sympy==1.13.1, but you have sympy 1.13.2 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install torch_sparse

  Using cached torch_sparse-0.6.18.tar.gz (209 kB)
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.9/29.9 MB 6.6 MB/s eta 0:00:00a 0:00:01
  Created wheel for torch_sparse: filename=torch_sparse-0.6.18-cp310-cp310-macosx_11_0_arm64.whl size=488911 sha256=7fd70217beac5145f727a90a6e2aad5d8cc2abf43f2a2801e247d7594f0ae821
  Stored in directory: /Users/myradeng/Library/Caches/pip/wheels/c9/dd/0f/a6a16f9f3b0236733d257b4b4ea91b548b984a341ed3b8f38c
Successfully built torch_sparse
Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install torch_scatter

  Using cached torch_scatter-2.1.2.tar.gz (108 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for torch_scatter: filename=torch_scatter-2.1.2-cp310-cp310-macosx_11_0_arm64.whl size=343606 sha256=fbf70105cf15b770197f16ca82a8e112df28657e55379da3fbba240a5b845c54
  Stored in directory: /Users/myradeng/Library/Caches/pip/wheels/92/f1/2b/3b46d54b134259f58c8363568569053248040859b1a145b3ce
Successfully built torch_scatter
Note: you may need to restart the kernel to use updated packages.


In [7]:
import torch_geometric

In [14]:
from pytorch_geometric_temporal.torch_geometric_temporal.dataset import PemsBayDatasetLoader

In [15]:
# Load in the data 
import os
import numpy as np
import pandas as pd
import torch
from pytorch_geometric_temporal.torch_geometric_temporal.dataset import PemsBayDatasetLoader

# Initialize the loader and load the dataset
loader = PemsBayDatasetLoader()
dataset = loader.get_dataset(num_timesteps_in=12, num_timesteps_out=12)

Dataset Information:
--------------------------------------------------
Number of nodes (traffic sensors): 325
Number of node features: 2
Number of timesteps in input: 12
Number of timesteps in target: 12
Number of edges in the graph: 2694


In [17]:
from pytorch_geometric_temporal.torch_geometric_temporal.signal import temporal_signal_split
train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.8)

In [19]:
train_dataset

In [27]:
class RecurrentGCN(torch.nn.Module):
    def __init__(self, node_features):
        super(RecurrentGCN, self).__init__()
        self.recurrent = DCRNN(in_channels=node_features, out_channels=32, K=1)
        self.linear = torch.nn.Linear(32, 1)

    def forward(self, x, edge_index, edge_weight):
        h = self.recurrent(x.flatten(start_dim=1), edge_index, edge_weight)
        h = F.relu(h)
        h = self.linear(h)
        return h
        
model = RecurrentGCN(node_features = 24)

In [29]:
for time, snapshot in enumerate(test_dataset):
    print(snapshot.x.shape, snapshot.edge_index.shape, snapshot.edge_attr.shape, snapshot.y.shape)
    break

torch.Size([325, 2, 12]) torch.Size([2, 2694]) torch.Size([2694]) torch.Size([325, 2, 12])


In [26]:
import torch
import torch.nn.functional as F
from pytorch_geometric_temporal.torch_geometric_temporal.nn.recurrent import DCRNN
from pytorch_geometric_temporal.torch_geometric_temporal.signal import temporal_signal_split

try:
    from tqdm import tqdm
except ImportError:
    def tqdm(iterable):
        return iterable

# Load and examine data
loader = PemsBayDatasetLoader()
dataset = loader.get_dataset(num_timesteps_in=12, num_timesteps_out=12)
#train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.8)
train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.2)

class RecurrentGCN(torch.nn.Module):
    def __init__(self, node_features):
        super(RecurrentGCN, self).__init__()
        self.recurrent = DCRNN(in_channels=node_features, out_channels=32, K=1)
        self.linear = torch.nn.Linear(32, 1)

    def forward(self, x, edge_index, edge_weight):
        h = self.recurrent(x.flatten(start_dim=1), edge_index, edge_weight)
        h = F.relu(h)
        h = self.linear(h)
        return h
        
model = RecurrentGCN(node_features = 24)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

model.train()

for epoch in tqdm(range(200)):
    cost = 0
    for time, snapshot in enumerate(train_dataset):
        y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr)
        cost = cost + torch.mean((y_hat-snapshot.y)**2)
    cost = cost / (time+1)
    cost.backward()
    optimizer.step()
    optimizer.zero_grad()
    
model.eval()
cost = 0
for time, snapshot in enumerate(test_dataset):
    y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr)
    cost = cost + torch.mean((y_hat-snapshot.y)**2)
cost = cost / (time+1)
cost = cost.item()
print("MSE: {:.4f}".format(cost))

# Evaluate
# model.eval()
# test_loss = 0
# predictions = []
# actuals = []

# with torch.no_grad():
#     for batch_x, batch_y in test_loader:
#         # Forward pass
#         y_hat = model(batch_x, test_edge_index, test_edge_attr)
#         y_hat = y_hat.view(batch_y.shape)
        
#         # Compute loss
#         loss = F.mse_loss(y_hat, batch_y)
#         test_loss += loss.item()
        
#         predictions.append(y_hat)
#         actuals.append(batch_y)

# avg_test_loss = test_loss / len(test_loader)
# print(f'\nTest Loss: {avg_test_loss:.4f}')

# # Plot predictions vs actuals for first test batch
# def plot_predictions(pred, actual, sensor_idx=0):
#     pred = pred[0, sensor_idx, 0, :].cpu().numpy()
#     actual = actual[0, sensor_idx, 0, :].cpu().numpy()
    
#     plt.figure(figsize=(10, 6))
#     plt.plot(pred, 'r-', label='Predicted')
#     plt.plot(actual, 'b-', label='Actual')
#     plt.title(f'Predictions vs Actuals for Sensor {sensor_idx}')
#     plt.legend()
#     plt.grid(True)
#     plt.show()

# # Plot results for first sensor
# plot_predictions(predictions[0], actuals[0])

  0%|          | 0/200 [00:00<?, ?it/s]


RuntimeError: The size of tensor a (325) must match the size of tensor b (2) at non-singleton dimension 1